In [1]:
import spacy
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from collections import defaultdict

In [54]:
# TO DO
# generados en pre y word2vec
# lematizar ?
# faltaria la reduccion de dimensionalidad ?

**Preprocesamiento del corpus**

In [52]:
def preProcessing(path_corpus):
    """
    input: pathfile corpus
    output: tokenizer doc, without stop-words and DIGITO word
    """
    
    nlp = spacy.load('es')
    # pipeline spacy
    pipe = ["tagger", "parser", "ner"]
    
    tokenizer_doc = []
    with open(path_corpus) as fd:
        for document in fd.readlines():
            document = document.strip()
            nlp.max_length = max(len(document), nlp.max_length)
            
            # nlp create doc object(token sequence)
            # disable pipeline from spacy
            tokens = [ token.text for token in nlp(document, disable=pipe) 
                       if not ((token.is_stop) or (token.text == 'DIGITO'))]
            
            tokens = [token.lower() for token in tokens]
            
            tokenizer_doc.append(tokens)
    return tokenizer_doc

**Vectorizar**

In [51]:
tokenizers = preProcessing("../corpus/xac")

In [57]:
def vector_model(tokenizers):
    """
    Word-embbeding with word2vec
    """
    # setting model
    sz = 300 # vector dimesion
    ww = 6   # size windows
    mc = 10  # ignores all word with total freq lower than this
    sg = 0   # skip-gram(1) or CBMWON(0)
    w2v_model = Word2Vec(tokenizers, window=ww, size=sz, min_count=mc, sg=sg)
    return w2v_model

In [61]:
w2v_model = vector_model(tokenizer)
vectors = w2v_model.wv.vectors

**Clustering**

In [62]:
# Configuracion de parametros
n_clusters = 250
max_iter = 400

kmeans = KMeans(n_clusters=n_clusters, max_iter=max_iter)

In [63]:
# Cada elemento se le asigna un label
kmeans.fit(vectors)
v_labels = kmeans.predict(vectors)

In [64]:
word_centroid = dict(zip(w2v_model.wv.index2word, v_labels))

dic = defaultdict(set)
for word, centroid in word_centroid.items():
    dic[centroid].add(word)

In [65]:
dic

defaultdict(set,
            {58: {'\n'},
             235: {'educación',
              'ejército',
              'empresa',
              'fuerzas',
              'imperio',
              'objetivo',
              'resto',
              'social',
              'sociales',
              'tuvieron',
              'y',
              'único'},
             85: {'a'},
             124: {'familia'},
             137: {'o'},
             71: {'especie'},
             218: {'año'},
             57: {'años'},
             141: {'orden'},
             35: {'ubicado'},
             101: {'arácnido'},
             24: {'pseudoscorpionida'},
             239: {'población'},
             148: {'ciudad'},
             39: {'if'},
             46: {'the'},
             7: {'none'},
             48: {'municipio'},
             72: {'medios'},
             18: {'habitantes'},
             29: {'ingresos'},
             114: {'localidad'},
             52: {'estadounidense'},
             138: {'nombre'